## Import Libraries

In [1]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
plt.ion()

In [3]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [4]:
from os import path, pardir

## Compile and program project

In [5]:
project_path = [path.curdir, "Dijkstra-S"]
project_path

['.', 'Dijkstra-S']

In [113]:
atprogram(path.abspath(path.join(*project_path)), verbose=2)

Building file: .././main.c
Invoking: ARM/GNU C Compiler : 6.3.1
"C:\Program Files (x86)\Atmel\Studio\7.0\toolchain\arm\arm-gnu-toolchain\bin\arm-none-eabi-gcc.exe"  -x c -mthumb -D__SAML11E16A__ -DDEBUG  -I"C:\Program Files (x86)\Atmel\Studio\7.0\Packs\arm\cmsis\5.0.1\CMSIS\Include" -I"../Config" -I".." -I"../examples" -I"../hal/include" -I"../hal/utils/include" -I"../hpl/core" -I"../hpl/crya" -I"../hpl/dmac" -I"../hpl/gclk" -I"../hpl/mclk" -I"../hpl/osc32kctrl" -I"../hpl/oscctrl" -I"../hpl/pm" -I"../hpl/port" -I"../hri" -I"../trustzone" -I"C:\Program Files (x86)\Atmel\Studio\7.0\Packs\Atmel\SAML11_DFP\1.0.91\include" -I"../../../../shared"  -O1 -ffunction-sections -mlong-calls -g3 -Wall -mcpu=cortex-m23 -c -std=gnu99 -mcmse -MD -MP -MF "main.d" -MT"main.d" -MT"main.o"   -o "main.o" ".././main.c" 
Finished building: .././main.c
Building target: Dijkstra-S.elf
Invoking: ARM/GNU Linker : 6.3.1
"C:\Program Files (x86)\Atmel\Studio\7.0\toolchain\arm\arm-gnu-toolchain\bin\arm-none-eabi-gcc.

0

## Data Logging

In [114]:
live_plot = True

Create a figure for the plot.

In [115]:
if live_plot:
    fig = plt.figure(figsize=(10, 6))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [116]:
config_dict = {
    "loggers": [LOGGER_OBJECT, LOGGER_CSV],
    "file_name_base": "experiment_dijkstra"
}
config_dict_plot = {
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "plot_pins_method": "line",
    "plot_xmax": 0.6,
    "window_title": "Experiment Dijkstra",
}

Stop criteria to pass to the logger:

In [117]:
def stop_fn(logger_data):
    return len(logger_data.gpio) and all(logger_data.gpio.values[-1])

Stop criteria to pass to the parser:

In [118]:
def stop_function(pin_values):
    return all(pin_values)

In [123]:
repetitions = 100

Perform the measurement.

In [ ]:
data = []
cd = config_dict.copy()
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()
    
    cd.update(config_dict_plot)
    cd["fig"] = fig
        
charges_graph = []
charges_dijkstra = []
times_graph = []
times_dijkstra = []

with DGILibExtra(**cd) as dgilib:
    for _ in range(repetitions):
        dgilib.device_reset()
        dgilib.logger.log(2, stop_fn)
        charge_graph, time_graph = power_and_time_per_pulse(dgilib.data, 3, stop_function=stop_function)
        charge_dijkstra, time_dijkstra = power_and_time_per_pulse(dgilib.data, 2, stop_function=stop_function)
        charges_graph += charge_graph
        times_graph += time_graph
        charges_dijkstra += charge_dijkstra
        times_dijkstra += time_dijkstra
        if len(charge_graph) != 1 or len(time_graph) != 1 or len(charge_dijkstra) != 1 or len(time_dijkstra) != 1:
            print(f"Parsing of measurement data failed, charge_graph:{charge_graph}, time_graph:{time_graph}, charge_dijkstra:{charge_dijkstra}, time_dijkstra:{time_dijkstra}")
            break
        dgilib.empty_data()
    
    dgilib.logger.plotobj.ax.set_title(f"Average of {repetitions} samples: charge: {sum(charges)/len(charges)*1e3:.06} mC, time: {sum(times)/len(times):.06} s")

In [34]:
import json

config = {}
config["name"] = "Dijkstra"
config["project_paths"] = [project_path]
config["config_dict"] = config_dict
config["config_dict_plot"] = config_dict_plot
config["analysis"] = {"pins":{2: ["Dijkstra"], 3: ["Create Graph"]}, 
                      "result_types": ["Charge", "Time"], 
                      "section_types": {"init": ["Create Graph"],
                                        "loop": ["Dijkstra"],
                                        "exit": []}}

with open("repeated_experiment.json", 'w') as config_file:  
    json.dump(config, config_file, indent=4)

## Generate test data

In [37]:
import numpy as np

In [90]:
# https://rosettacode.org/wiki/Dijkstra%27s_algorithm#Python

from collections import namedtuple, deque
 
 
inf = float('inf')
Edge = namedtuple('Edge', 'start, end, cost')
 
class Graph():
    def __init__(self, edges):
        self.edges = edges2 = [Edge(*edge) for edge in edges]
        self.vertices = set(sum(([e.start, e.end] for e in edges2), []))
 
    def dijkstra(self, source, dest):
        assert source in self.vertices
        dist = {vertex: inf for vertex in self.vertices}
        previous = {vertex: None for vertex in self.vertices}
        dist[source] = 0
        q = self.vertices.copy()
        neighbours = {vertex: set() for vertex in self.vertices}
        for start, end, cost in self.edges:
            neighbours[start].add((end, cost))
        # print(neighbours)
 
        while q:
            u = min(q, key=lambda vertex: dist[vertex])
            q.remove(u)
            if dist[u] == inf or u == dest:
                break
            for v, cost in neighbours[u]:
                alt = dist[u] + cost
                if alt < dist[v]: # Relax (u,v,a)
                    dist[v] = alt
                    previous[v] = u
        # print(previous)
        # print(neighbours)
        s, u = deque(), dest
        while previous[u]:
            s.appendleft(u)
            u = previous[u]
        s.appendleft(u)
        return s

In [91]:
class CGraph(Graph):
    def print_c(self):
        print('int main (void) {\n\tgraph_t *g = calloc(1, sizeof (graph_t));')
        for edge in self.edges:
            print(f'\tadd_edge(g, {edge.start}, {edge.end}, {edge.cost});')
        end_node = len(self.vertices) - 1
        print('\tdijkstra(g, 0, ' + f'{end_node}' + ');\n#ifdef DEBUG_PRINT\n\tprint_path(g, ' + f'{end_node}' + ');\n#endif // DEBUG_PRINT\n\treturn 0;\n}')
        
    def print_path(self):
        path = self.dijkstra(0, len(self.vertices) - 1)
        s = f'hops: {len(path)}, path: '
        for e in reversed(path):
            s += f'{e}<-'
        print(s + '0')

def random_edges(nodes, nedges):
    idx = np.random.choice(np.prod((nodes,nodes-1)), nedges, replace=False)
    diags = np.array([(i + i * (nodes - 1)) for i in range(nodes)])
    return np.vstack(np.unravel_index(idx + np.searchsorted(diags, idx+1), (nodes,nodes))).T

def random_graph(nodes, nedges):
    edges = random_edges(nodes, nedges)
    graph = CGraph([(*edge, np.random.randint(int_max+1)) for edge in edges])
    return graph

In [104]:
np.random.seed(314)
nodes = 128
nedges = 512  # 358 with 256 nodes and seed 314 has 29 hops
int_max = 255
graph = random_graph(nodes, nedges)

In [105]:
graph.print_path()
graph.print_c()

hops: 4, path: 127<-84<-72<-116<-0
int main (void) {
	graph_t *g = calloc(1, sizeof (graph_t));
	add_edge(g, 25, 69, 197);
	add_edge(g, 56, 50, 197);
	add_edge(g, 15, 107, 36);
	add_edge(g, 26, 79, 103);
	add_edge(g, 39, 104, 103);
	add_edge(g, 4, 92, 224);
	add_edge(g, 10, 49, 17);
	add_edge(g, 19, 73, 49);
	add_edge(g, 82, 15, 39);
	add_edge(g, 9, 115, 239);
	add_edge(g, 113, 31, 157);
	add_edge(g, 48, 44, 22);
	add_edge(g, 15, 118, 125);
	add_edge(g, 67, 5, 248);
	add_edge(g, 73, 53, 189);
	add_edge(g, 97, 43, 59);
	add_edge(g, 0, 124, 86);
	add_edge(g, 72, 84, 0);
	add_edge(g, 5, 59, 125);
	add_edge(g, 107, 13, 12);
	add_edge(g, 16, 8, 37);
	add_edge(g, 25, 32, 153);
	add_edge(g, 73, 66, 250);
	add_edge(g, 67, 30, 244);
	add_edge(g, 18, 108, 54);
	add_edge(g, 72, 38, 146);
	add_edge(g, 109, 8, 32);
	add_edge(g, 74, 69, 227);
	add_edge(g, 58, 86, 108);
	add_edge(g, 116, 29, 118);
	add_edge(g, 115, 57, 75);
	add_edge(g, 99, 32, 142);
	add_edge(g, 84, 127, 91);
	add_edge(g, 49, 73, 12

## Analysis

In [29]:
import pandas as pd

In [30]:
df = pd.DataFrame({'times_graph': times_graph, 'charges_graph':charges_graph, 'times_dijkstra': times_dijkstra, 'charges_dijkstra':charges_dijkstra})

In [31]:
df

,times_graph,charges_graph,times_dijkstra,charges_dijkstra
0,0.013002,0.000005,0.00264,9.497056e-07
1,0.013001,0.000005,0.00264,9.463899e-07
2,0.013003,0.000005,0.00264,9.506094e-07
3,0.013003,0.000005,0.00264,9.424458e-07
4,0.013003,0.000005,0.00264,9.400561e-07
5,0.013001,0.000005,0.00264,9.456012e-07
6,0.013003,0.000005,0.00264,9.481511e-07
7,0.013003,0.000005,0.00264,9.506094e-07
8,0.012936,0.000005,0.00264,9.440007e-07
9,0.013002,0.000005,0.00264,9.456088e-07


In [32]:
df.describe()

,times_graph,charges_graph,times_dijkstra,charges_dijkstra
count,10.000000,1.000000e+01,1.000000e+01,1.000000e+01
mean,0.012996,4.919830e-06,2.640081e-03,9.463178e-07
std,0.000021,8.892419e-09,1.772936e-07,3.531844e-09
min,0.012936,4.903446e-06,2.639829e-03,9.400561e-07
25%,0.013002,4.917448e-06,2.639925e-03,9.444008e-07
50%,0.013002,4.921841e-06,2.640085e-03,9.459993e-07
75%,0.013003,4.926007e-06,2.640261e-03,9.493169e-07
max,0.013003,4.930964e-06,2.640277e-03,9.506094e-07
